In [12]:
import os
import sys
import yaml
from typing import (
    Dict,
    List, 
    Optional,
    Tuple,
    Union
)

In [2]:
__file__ = os.path.join(os.getcwd(),"cs_wrap1.ipynb")

In [3]:
mod_path = os.path.join(os.path.dirname(os.path.abspath(__file__)),"..")

In [4]:
sys.path.append(mod_path)

In [5]:
from convert_source.utils.bids_info import (
    construct_bids_dict,
    construct_bids_name
)

In [26]:
from convert_source.utils.utils import (
    BIDSimg,
    collect_info,
    convert_image_data,
    SubInfoError,
    SubDataInfo,
)

In [7]:
from convert_source.utils.const import DEFAULT_CONFIG

In [10]:
def read_config(config_file: Optional[str] = "", 
                verbose: Optional[bool] = False
                ) -> Tuple[Dict,List[str],Dict]:
    '''Reads configuration file and creates a dictionary of search terms for 
    certain modalities provided that BIDS modalities are used as keys. If
    exclusions are provided (via the key 'exclude') then an exclusion list is 
    created. Otherwise, 'exclusion_list' is returned as an empty list. If 
    additional settings are specified, they should be done so via the key
    'metadata' to enable writing of additional metadata. Otherwise, an 
    empty dictionary is returned.

    BIDS modalities:
        - anat:
            - T1w, T2w, FLAIR, etc.
        - func:
            - task:
                - resting state, <task-name>
        - dwi
        - fmap

    Usage example:
        >>> [search_dict, exclusion_list, meta_dict] = read_config(config_file)
    
    Arguments:
        config_file: File path to yaml configuration file. If no file is used, then the default configuration file is used.
        verbose: Prints additional information to screen.
    
    Returns: 
        search_dict: Nested dictionary of search terms for BIDS modalities.
        exclusion_list: List of exclusion terms.
        meta_dict: Nested dictionary of metadata terms to write to JSON file(s).
    
    Raises:
        ConfigFileReadError: Error that arises if no heuristic search terms are provided.
    '''
    class ConfigFileReadError(Exception):
        pass

    if config_file:
        config_file: str = os.path.abspath(config_file)
    else:
        config_file: str = DEFAULT_CONFIG

    with open(config_file) as file:
        data_map = yaml.safe_load(file)
        if verbose:
            print("Initialized parameters from configuration file")
    
    # Required search terms
    if any("search" in data_map for element in data_map):
        if verbose:
            print("Categorizing search terms")
        search_dict = data_map["search"]
        del data_map["search"]
    else:
        if verbose:
            print("Heuristic search terms required. Exiting...")
        raise ConfigFileReadError("Heuristic search terms required. Exiting...")

    # Exclusion terms  
    if any("exclude" in data_map for element in data_map):
        if verbose:
            print("Exclusion option implemented")
        exclusion_list = data_map["exclude"]
        del data_map["exclude"]
    else:
        if verbose:
            print("Exclusion option not implemented")
        exclusion_list = list()
    
    # Metadata terms
    if any("metadata" in data_map for element in data_map):
        if verbose:
            print("Including additional settings for metadata")
        meta_dict = data_map["metadata"]
        del data_map["metadata"]
    else:
        if verbose:
            print("No metadata settings")
        meta_dict = dict()
        
    return search_dict,exclusion_list,meta_dict

In [13]:
[search_dict,exclusion_list,meta_dict] = read_config()

In [17]:
# search_dict
# exclusion_list
# meta_dict

In [18]:
par_img_dir = "test.img"

In [19]:
par_img_dir = os.path.abspath(par_img_dir)

In [27]:
subs_data = collect_info(parent_dir=par_img_dir,
                        exclusion_list=exclusion_list)

In [43]:
subs_data[90].ses

'001'

In [ ]:
# TODO:
#   - Write function that takes input (of array) of SubDataInfo object(s) {sub,data,ses}, and search dict - to search and categorize data
#   - helper/support function to aid in search and categorization of data.

In [31]:
# Pass the large number of args as a dict
# 
# Needs to run within for-loop
# 
# Unable to test without actual data
# 
# img_data = convert_image_data(file=subs_data[0].data,
#                                 basename="random_str",
#                                 out_dir="out_dir")